# 1. Professionally Operated Weather Stations (PRWS)
This notebook provides a comprehensive data preprocessing workflow for Professionally Operated Weather Stations (PRWS) and Wunderground.
The meteorological observations used in this study were obtained from the Weather Observations Website (WOW), operated by the UK Met Office.  
Data were downloaded from: https://wow.metoffice.gov.uk/support?category=faq 

**Weather Stations Used in This Study**

| Weather station         | Station code | Elevation (m) | Latitude / Longitude |
|------------------------|--------------|---------------|----------------------|
| Laverton RAAF          | 087031       | 20.1          | -37.8565 / 144.7565  |
| Melbourne Olympic Park | 086338       | 7.5           | -37.8255 / 144.9816  |
| Essendon Airport       | 086038       | 78.4          | -37.7276 / 144.9066  |
| Melbourne Airport      | 086282       | 113.4         | -37.6654 / 144.8322  |
| Viewbank               | 086068       | 66.1          | -37.7408 / 145.0972  |
| Moorabbin Airport      | 086077       | 15.24         | -37.9800 / 145.0960  |

- **Mean Sea-Level Pressure (mslp)**: Atmospheric pressure reduced to mean sea level (hPa)
- **Relative Humidity (rh)**: Relative humidity (%)
- **Rainfall Accumulation (rain)**: Accumulated precipitation (mm)
- **Wind Speed (ws_ms)**: Wind speed (m/s), the initial data were in knots.
- **Wind Direction (wd)**: Wind direction (degrees)


## 1.1. Merge all data and preview the raw data

In [ ]:
import pandas as pd
import glob
import os

# Set your folder path inmput
folder_path = r"C:\Users\percy\Documents\studies\visual_studio_code\PhD-chapter1a-prws-cws-data-preprocessing\data\bom\raw"

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Empty list to store DataFrames
merged_list = []

# Loop through each CSV file
for file in csv_files:
    # Read the CSV
    df = pd.read_csv(file)

    # Add file name column
    df['file_name'] = os.path.basename(file)

    # Add to merge list
    merged_list.append(df)

# Combine all into one DataFrame
merged_df = pd.concat(merged_list, ignore_index=True)

# Preview first 5 rows
display(merged_df.head(5))

,Id,Site Id,Longitude,Latitude,Report Date / Time,Air Temperature,Wet Bulb,Dew Point,Concrete Temp.,Grass Temp.,...,v1_Flood,v1_Ice,v1_Landslide,v1_Lightning,v1_PoorVisibility,v1_Snow,v1_Wildfire,v1_Wind,v1_Other,file_name
0,﻿20221201wyfxf9utnce65f7oyyb96sm4ma,429976050,144.906601,-37.7276,2022-12-01 00:50:00,15.7,NaN,12.5,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ea_dec.csv
1,20221201brcdfa5tdoe65f7oyyb96sm4ma,429976050,144.906601,-37.7276,2022-12-01 01:50:00,16.6,NaN,11.7,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ea_dec.csv
2,20221201k3e6175troe65f7oyyb96sm4se,429976050,144.906601,-37.7276,2022-12-01 02:50:00,18.0,NaN,12.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ea_dec.csv
3,20221201b4bw78mtfwe65f7oyyb96sm4ma,429976050,144.906601,-37.7276,2022-12-01 03:50:00,19.0,NaN,12.3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ea_dec.csv
4,20221201jqgz4dutgwe65f7oyyb96sm4se,429976050,144.906601,-37.7276,2022-12-01 04:50:00,18.8,NaN,12.1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ea_dec.csv


## 1.2. Clean the merged data

In [20]:

from datetime import timedelta

#Change a column named "Report Date / Time" into "Time"
merged_df_clean = merged_df.rename(columns={'Report Date / Time': 'Time'})

merged_df_clean['Time'] = pd.to_datetime(merged_df_clean['Time'], format="%Y-%m-%d %H:%M:%S", errors='coerce')

# Shift 10 minutes forward so 00:50 → 01:00
merged_df_clean['Time_UTC'] = merged_df_clean['Time'] + timedelta(minutes=10)

# Round to hour
merged_df_clean['Time_UTC'] = merged_df_clean['Time_UTC'].dt.floor('h')

# Subtract 11 hours and create a new column 'Time'
merged_df_clean['Time_AEDT'] = merged_df_clean['Time_UTC'] + pd.Timedelta(hours=11)

#remove the extension on the file name
merged_df_clean['file_name'] = merged_df_clean['file_name'].str[:-8]

# Rename columns
rename_map = {
    'Mean Sea-Level Pressure': 'mslp',
    'Air Temperature': 'ta',
    'Relative Humidity': 'rh',
    'Rainfall Accumulation': 'rain',
    'Wind Speed': 'ws_kn',
    'Wind Direction': 'wd'
}
merged_df_clean = merged_df_clean.rename(columns=rename_map)

#Wind speed from knot to m/s
merged_df_clean['ws_ms'] = merged_df_clean['ws_kn'] * 0.514444

# Select only needed columns (without ws_kn now)
columns_to_keep = ['Time', 'file_name', 'Time_UTC', 'Time_AEDT', 'ws_ms'] + [col for col in rename_map.values() if col != 'ws_kn']
merged_df_clean = merged_df_clean[columns_to_keep]

# Preview first 5 rows
display(merged_df_clean.head(5))

,Time,file_name,Time_UTC,Time_AEDT,ws_ms,mslp,ta,rh,rain,wd
0,2022-12-01 00:50:00,ea,2022-12-01 01:00:00,2022-12-01 12:00:00,5.144440,1018.5,15.7,81,0.0,157.5
1,2022-12-01 01:50:00,ea,2022-12-01 02:00:00,2022-12-01 13:00:00,7.202216,1018.2,16.6,73,0.0,157.5
2,2022-12-01 02:50:00,ea,2022-12-01 03:00:00,2022-12-01 14:00:00,7.202216,1017.9,18.0,69,0.0,157.5
3,2022-12-01 03:50:00,ea,2022-12-01 04:00:00,2022-12-01 15:00:00,7.202216,1017.5,19.0,65,0.0,157.5
4,2022-12-01 04:50:00,ea,2022-12-01 05:00:00,2022-12-01 16:00:00,7.202216,1017.4,18.8,65,0.0,157.5


## 1.3. Fill missing values

Load the dataset and perform exploratory data analysis including shape, data types, statistical summaries, and identifying missing values.

In [22]:
# Sort by file_name and Time_AEDT
merged_df_clean = merged_df_clean.sort_values(['file_name', 'Time_AEDT']).reset_index(drop=True)

# Columns that need interpolation
columns_to_interp = ['ta', 'rh', 'ws_ms', 'wd', 'mslp', 'rain']
filled_dfs = []

for station, group in merged_df_clean.groupby('file_name'):
    group = group.drop_duplicates(subset='Time_AEDT')
    group = group.set_index('Time_AEDT')

    # Create full hourly index
    full_index = pd.date_range(start=group.index.min(), end=group.index.max(), freq='1h')

    # Insert missing time rows
    missing_times = full_index.difference(group.index)
    missing_df = pd.DataFrame(index=missing_times)
    missing_df['file_name'] = station

    # Combine and sort
    combined = pd.concat([group, missing_df]).sort_index()
    combined = combined.asfreq('h')  # set hourly frequency for time interpolation

    # Filter existing columns
    available_cols = [col for col in columns_to_interp if col in combined.columns]

    # Convert to numeric
    for col in available_cols:
        combined[col] = pd.to_numeric(combined[col], errors='coerce')

    # Interpolate only if at least some data exists
    if available_cols:
        combined[available_cols] = combined[available_cols].interpolate(method='time', limit_direction='both')

    filled_dfs.append(combined)

# Final result
merged_df_filled = pd.concat(filled_dfs).reset_index().rename(columns={'index': 'Time_AEDT'})

# Optional: check how many rows were added
print(f"Original rows: {len(merged_df_clean)}, After fill: {len(merged_df_filled)}")

# Preview first 5 rows
display(merged_df_filled.head(5))

Original rows: 13030, After fill: 13104


,Time_AEDT,Time,file_name,Time_UTC,ws_ms,mslp,ta,rh,rain,wd
0,2022-11-30 12:00:00,2022-11-30 00:50:00,ea,2022-11-30 01:00:00,3.086664,1019.2,17.2,59.0,0.0,202.5
1,2022-11-30 13:00:00,2022-11-30 01:50:00,ea,2022-11-30 02:00:00,2.572220,1018.6,17.3,57.0,0.0,180.0
2,2022-11-30 14:00:00,2022-11-30 02:50:00,ea,2022-11-30 03:00:00,6.173328,1018.1,17.7,56.0,0.0,157.5
3,2022-11-30 15:00:00,2022-11-30 03:50:00,ea,2022-11-30 04:00:00,6.687772,1017.6,18.5,52.0,0.0,157.5
4,2022-11-30 16:00:00,2022-11-30 04:50:00,ea,2022-11-30 05:00:00,7.202216,1017.0,18.6,52.0,0.0,202.5


## 1.4. Join with the BOM metadata

In [ ]:
# load BOM metadata
bom_metadata = pd.read_csv(r"C:\Users\percy\Documents\studies\visual_studio_code\PhD-chapter1a-prws-cws-data-preprocessing\data\bom\bom_metadata.csv")

# Preview first 5 rows
display(bom_metadata.head(5))

# Merge metadata with filled data
bom_final_df = pd.merge(merged_df_filled, bom_metadata, on='file_name', how='left')

# Keep and reorder relevant columns
bom_final_df = bom_final_df[["p_id", "station", "Time_UTC", "Time_AEDT", "ta", "rh", "ws_ms", "wd", "rain", "mslp"]]

# Preview first 5 rows
display(bom_final_df.head(5))

,file_name,p_id,station,station_code,alt_m,latitude,longitude
0,ea,200,Essendon Airport,86038,78.4,-37.7276,144.9066
1,la,600,Laverton RAAF,87031,20.1,-37.8565,144.7565
2,ma,400,Melbourne Airport,86282,113.4,-37.6654,144.8322
3,mo,500,Melbourne Olympic Park,86338,7.5,-37.8255,144.9816
4,mor,700,Moorabbin Airport,86077,66.1,-37.7408,145.0972


,p_id,station,Time_UTC,Time_AEDT,ta,rh,ws_ms,wd,rain,mslp
0,200,Essendon Airport,2022-11-30 01:00:00,2022-11-30 12:00:00,17.2,59.0,3.086664,202.5,0.0,1019.2
1,200,Essendon Airport,2022-11-30 02:00:00,2022-11-30 13:00:00,17.3,57.0,2.572220,180.0,0.0,1018.6
2,200,Essendon Airport,2022-11-30 03:00:00,2022-11-30 14:00:00,17.7,56.0,6.173328,157.5,0.0,1018.1
3,200,Essendon Airport,2022-11-30 04:00:00,2022-11-30 15:00:00,18.5,52.0,6.687772,157.5,0.0,1017.6
4,200,Essendon Airport,2022-11-30 05:00:00,2022-11-30 16:00:00,18.6,52.0,7.202216,202.5,0.0,1017.0


# 2. Wunderground
## 2.1. Merge all data
The data were accessed via the following website:  
https://www.wunderground.com/wundermap

Observations from **1 December 2022 to 28 February 2023** were downloaded for **195 stations**.  
Stations were **manually selected** based on data availability during this period.

The complete dataset was downloaded using the following code repository:  
https://github.com/Karlheinzniebuhr/the-weather-scraper

Data are generally collected every 5 minutes with some exception of every 15 minutes.

In [ ]:
# Set your folder path inmput
folder_path = r"C:\Users\percy\Documents\studies\visual_studio_code\PhD-chapter1a-prws-cws-data-preprocessing\data\wunderground\raw"

# Get all CSV files in the folder
wunderground_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Empty list to store DataFrames
wunderground_list = []

# Loop through each CSV file
for file in wunderground_files:
    # Read the CSV
    df = pd.read_csv(file)

    # Add file name column
    df['file_name'] = os.path.basename(file)

    # Add to merge list
    wunderground_list.append(df)

# Combine all into one DataFrame
wunderground_merged_df = pd.concat(wunderground_list, ignore_index=True)

# Combine Date and Time columns into a single datetime string
wunderground_merged_df['datetime_str'] = wunderground_merged_df['Date'].astype(str).str.strip() + " " + wunderground_merged_df['Time'].astype(str).str.strip()

# Parse the datetime string
wunderground_merged_df['datetime'] = pd.to_datetime(wunderground_merged_df['datetime_str'], format="%Y/%m/%d %I:%M %p", errors='coerce')

# Round full datetime to nearest hour
wunderground_merged_df['rounded_hour'] = wunderground_merged_df['datetime'].dt.round('h')

# Compute absolute time difference to the rounded hour
wunderground_merged_df['time_diff'] = (wunderground_merged_df['datetime'] - wunderground_merged_df['rounded_hour']).abs()

#remove the extension on the file name
wunderground_merged_df['file_name'] = wunderground_merged_df['file_name'].str[:-4]

# Keep only the row closest to each rounded hour
wunderground_nearest_df = wunderground_merged_df.loc[
    wunderground_merged_df.groupby(['file_name', 'rounded_hour'])['time_diff'].idxmin()
].reset_index(drop=True)

# Rename columns
wunderground_clean_df = wunderground_nearest_df.rename(columns={
    "Temperature_C": "ta",
    "Humidity_%": "rh",
    "Speed_kmh": "ws_kmh",
    "Wind": "wd",
    "Pressure_hPa": "mslp",
    "Precip_Accum_mm": "rain",
    "datetime": "datetime",
    "rounded_hour": "Time_AEDT",
    "file_name": "file_name"
})

# Keep only the desired columns in the desired order
wunderground_clean_df = wunderground_clean_df[["file_name", "datetime", "Time_AEDT", "ta", "rh", "ws_kmh", "wd", "rain", "mslp"]]

print(f"✅ Done! {len(wunderground_clean_df)} rows saved — one per hour.")

# Preview first 5 rows
display(wunderground_clean_df.head(5))

✅ Done! 394001 rows saved — one per hour.


,file_name,datetime,Time_AEDT,ta,rh,ws_kmh,wd,rain,mslp
0,IDIGGE6,2023-01-03 14:59:00,2023-01-03 15:00:00,20.28,53.0,7.08,South,0.0,1011.85
1,IDIGGE6,2023-01-03 15:59:00,2023-01-03 16:00:00,20.89,52.0,8.21,South,0.0,1011.85
2,IDIGGE6,2023-01-03 16:59:00,2023-01-03 17:00:00,19.22,59.0,3.7,South,0.0,1011.85
3,IDIGGE6,2023-01-03 17:59:00,2023-01-03 18:00:00,18.39,64.0,7.56,South,0.0,1011.85
4,IDIGGE6,2023-01-03 18:59:00,2023-01-03 19:00:00,17.17,66.0,6.92,SSE,0.0,1011.85


## 2.2. Join with Wunderground Metadata
The metadata were obtained directly from the website by manually selecting each station.  
Station coordinates were digitised and extracted using **OpenStreetMap** in **QGIS**.

- **elevation**: Elevation reported in the station metadata  
- **z_GEE**: Elevation verified using Google Earth Pro  
- **latitude / longitude**: Latitude and longitude provided in the station metadata  
- **lat_cal / long_cal**: Latitude and longitude extracted from a manually digitised station shapefile


In [ ]:
# load Wunderground metadata
wunderground_metadata = pd.read_csv(r"C:\Users\percy\Documents\studies\visual_studio_code\PhD-chapter1a-prws-cws-data-preprocessing\data\wunderground\wunderground_metadata.csv")

# Preview first 5 rows
display(wunderground_metadata.head(5))

# Merge metadata with filled data
wunderground_final_df = pd.merge(wunderground_clean_df, wunderground_metadata, on='file_name', how='left')

#Reorder columns
wunderground_final_df = wunderground_final_df[[
    "p_id", "file_name", "datetime", "Time_AEDT",
    "ta", "rh", "ws_kmh", "wd", "rain", "mslp"
]]

# Preview first 5 rows
display(wunderground_final_df.head(5))

,p_id,file_name,station_name,latitude,longitude,elevation,hardware,software,lat_cal,long_cal,z_GEE
0,1,IMELBOUR613,Greensborough,37.708,145.097,85.0,Other,WeatherRanger V2.5.1,-37.7080,145.097,81.0
1,2,IMELBO3535,Breese Street,37.761,144.962,15.0,Davis Vantage Vue (Wireless),meteobridge,-37.7608,144.962,53.0
2,3,IMELBO4021,heatsweather,37.676,145.058,117.0,Other,NaN,-37.6760,145.058,108.0
3,4,IMELBO2098,Bar Crestwood,37.713,145.067,96.0,Other,NaN,-37.7129,145.067,115.0
4,5,IMELBO4107,Pantech WH2900,37.777,145.020,11.0,Ambient Weather WS-2090 (Wireless),EasyWeatherV1.6.9,-37.7770,145.020,36.0


,p_id,file_name,datetime,Time_AEDT,ta,rh,ws_kmh,wd,rain,mslp
0,113,IDIGGE6,2023-01-03 14:59:00,2023-01-03 15:00:00,20.28,53.0,7.08,South,0.0,1011.85
1,113,IDIGGE6,2023-01-03 15:59:00,2023-01-03 16:00:00,20.89,52.0,8.21,South,0.0,1011.85
2,113,IDIGGE6,2023-01-03 16:59:00,2023-01-03 17:00:00,19.22,59.0,3.7,South,0.0,1011.85
3,113,IDIGGE6,2023-01-03 17:59:00,2023-01-03 18:00:00,18.39,64.0,7.56,South,0.0,1011.85
4,113,IDIGGE6,2023-01-03 18:59:00,2023-01-03 19:00:00,17.17,66.0,6.92,SSE,0.0,1011.85
